In [37]:
import pandas as pd

features = pd.read_csv('fulldata_colonnes_essentielles_prepross.csv', index_col=0)
features = features.drop('peak_int', 1)
features.head(5)

,ACCESSION,class1,class2,class3,AC_mode,ion_precursor_mz,peak_mz,num_peak,peak_rel_int
7,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,1,0.140564,0.063692,0.122807,0.002695
8,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,1,0.140564,0.037912,0.122807,0.008985
9,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,1,0.140564,0.034229,0.122807,0.089847
10,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,1,0.140564,0.030329,0.122807,0.053908
11,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,1,0.140564,0.028596,0.122807,0.013477


In [38]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44524 entries, 7 to 49307
Data columns (total 9 columns):
ACCESSION           44524 non-null object
class1              44524 non-null object
class2              44524 non-null object
class3              44524 non-null object
AC_mode             44524 non-null int64
ion_precursor_mz    44524 non-null float64
peak_mz             44524 non-null float64
num_peak            44524 non-null float64
peak_rel_int        44524 non-null float64
dtypes: float64(4), int64(1), object(4)
memory usage: 3.4+ MB


In [39]:
target1 = features['class1']
target2 = features['class2']
target3 = features['class3']

drop = ['class1', 'class2', 'class3', 'ACCESSION']
data = features.drop(drop, axis =1)

In [40]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(data,target1, test_size=0.2, random_state=12)
X_train2, X_test2, y_train2, y_test2 = train_test_split(data,target2, test_size=0.2, random_state=12)
X_train3, X_test3, y_train3, y_test3 = train_test_split(data,target3, test_size=0.2, random_state=12)

In [41]:
from sklearn import ensemble

clf = ensemble.RandomForestClassifier(n_jobs = -1, random_state = 321)
clf.fit(X_train1, y_train1)
clf.fit(X_train2, y_train2)
clf.fit(X_train3, y_train3)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=321, verbose=0, warm_start=False)

In [42]:
y_pred1 = clf.predict(X_test1)
y_pred2 = clf.predict(X_test2)
y_pred3 = clf.predict(X_test3)
print('score représentant le taux de bonnes prédictions (accuracy) classe 1 :', clf.score(X_test1, y_test1))
print('score représentant le taux de bonnes prédictions (accuracy) classe 2 :', clf.score(X_test2, y_test2))
print('score représentant le taux de bonnes prédictions (accuracy) classe 3 :', clf.score(X_test3, y_test3))

score représentant le taux de bonnes prédictions (accuracy) classe 1 : 0.104435710275
score représentant le taux de bonnes prédictions (accuracy) classe 2 : 0.230881527232
score représentant le taux de bonnes prédictions (accuracy) classe 3 : 0.751712521056


In [17]:
y_probas1 = clf.predict_proba(X_test1)
y_probas2 = clf.predict_proba(X_test2)
y_probas3 = clf.predict_proba(X_test3)


In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold 

parameter_gridsearch = {
                 'max_depth' : [3, 4],  
                 'n_estimators': [50, 20],  
                 'max_features': ['sqrt', 'auto', 'log2'],      
                 'min_samples_split': [2],      
                 'min_samples_leaf': [1, 3, 4],
                 'bootstrap': [True, False],
                 }
randomforest = RandomForestClassifier()
crossvalidation = StratifiedKFold(y_train3 , n_folds=5)

gridsearch = GridSearchCV(randomforest,            
                               scoring='accuracy',
                               param_grid=parameter_gridsearch,
                               cv=crossvalidation)


gridsearch.fit(X_train3, y_train3)    
model = gridsearch
parameters = gridsearch.best_params_

D:\anaconda2\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


KeyboardInterrupt: 

In [33]:
print('Best Score: {}'.format(gridsearch.best_score_))

Best Score: 0.4567225357253151


In [34]:
parameters

{'bootstrap': False,
 'max_depth': 4,
 'max_features': 'log2',
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 20}

In [44]:
pd.crosstab(y_test3,y_pred3, rownames=['Actual Species'], colnames=['Predicted Species'])

Predicted Species,Abscisic acid,Acacetin,Acacetin glycoside,Acetaminophen,Acetic acid,Acetonic acid,Acetophenone,Acetyl choline,Acetyl phosphate,Aconitic acid,...,Yunantaxusin,Zearalenone,Zeatin,Zeaxanthin,Zygadenin,caffeic acid,ferulic acid,p-Coumaric acid,p-coumaric acid,tanshinone
Actual Species,,,,,,,,,,,,,,,,,,,,,
Abscisic acid,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acacetin,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acacetin glycoside,0,0,15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetaminophen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetic acid,0,0,0,0,8,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Acetonic acid,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetyl choline,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetyl phosphate,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Aconitic acid,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


In [45]:
import pandas as pd

features = pd.read_csv('fulldata_complete_selection_prepross.csv', index_col=0)
features.head(5)

,solvent_CH3CH/H2O/HCOOH,solvent_CH3CN/8mMCH3COONH4,solvent_CH3CN/H2O,solvent_CH3CN/H2O/CH3COOH,solvent_CH3CN/H2O/HCOOH,solvent_CH3OH,solvent_CH3OH/H2O/HCOOH,solvent_H2O/CH3CN,solvent_H2O/CH3CN+CH3OH/HCOOH/CH3COONH4,solvent_H2O/CH3CN/C4H11N,...,class3,AC_mode,exact_mass,formule,ion_precursor_mz,num_peak,peak_mz,peak_rel_int,peak_int,presence_solvent
7,0,0,0,0,0,0,0,0,0,0,...,Phenylpropanoid glycoside,1,0.000006,C15H22O6,0.140564,0.122807,0.063692,0.002695,0.0,1
8,0,0,0,0,0,0,0,0,0,0,...,Phenylpropanoid glycoside,1,0.000006,C15H22O6,0.140564,0.122807,0.037912,0.008985,0.0,1
9,0,0,0,0,0,0,0,0,0,0,...,Phenylpropanoid glycoside,1,0.000006,C15H22O6,0.140564,0.122807,0.034229,0.089847,0.0,1
10,0,0,0,0,0,0,0,0,0,0,...,Phenylpropanoid glycoside,1,0.000006,C15H22O6,0.140564,0.122807,0.030329,0.053908,0.0,1
11,0,0,0,0,0,0,0,0,0,0,...,Phenylpropanoid glycoside,1,0.000006,C15H22O6,0.140564,0.122807,0.028596,0.013477,0.0,1


In [46]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44524 entries, 7 to 49307
Columns: 1647 entries, solvent_CH3CH/H2O/HCOOH to presence_solvent
dtypes: float64(6), int64(1635), object(6)
memory usage: 559.8+ MB


In [47]:
target1 = features['class1']
target2 = features['class2']
target3 = features['class3']

drop = ['class1', 'class2', 'class3', 'ACCESSION', 'name', 'formule']
data = features.drop(drop, axis =1)

In [48]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(data,target1, test_size=0.2, random_state=12)
X_train2, X_test2, y_train2, y_test2 = train_test_split(data,target2, test_size=0.2, random_state=12)
X_train3, X_test3, y_train3, y_test3 = train_test_split(data,target3, test_size=0.2, random_state=12)

In [49]:
from sklearn import ensemble

clf = ensemble.RandomForestClassifier(n_jobs = -1, random_state = 321)
clf.fit(X_train1, y_train1)
clf.fit(X_train2, y_train2)
clf.fit(X_train3, y_train3)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=321, verbose=0, warm_start=False)

In [50]:
y_pred1 = clf.predict(X_test1)
y_pred2 = clf.predict(X_test2)
y_pred3 = clf.predict(X_test3)
print('score représentant le taux de bonnes prédictions (accuracy) classe 1 :', clf.score(X_test1, y_test1))
print('score représentant le taux de bonnes prédictions (accuracy) classe 2 :', clf.score(X_test2, y_test2))
print('score représentant le taux de bonnes prédictions (accuracy) classe 3 :', clf.score(X_test3, y_test3))

score représentant le taux de bonnes prédictions (accuracy) classe 1 : 0.124312184166
score représentant le taux de bonnes prédictions (accuracy) classe 2 : 0.283099382369
score représentant le taux de bonnes prédictions (accuracy) classe 3 : 0.930151600225


In [51]:
y_probas1 = clf.predict_proba(X_test1)
y_probas2 = clf.predict_proba(X_test2)
y_probas3 = clf.predict_proba(X_test3)

In [52]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_test3,y_pred))  
print(classification_report(y_test3,y_pred3)) 

                             precision    recall  f1-score   support

              Abscisic acid       1.00      1.00      1.00         7
                   Acacetin       1.00      1.00      1.00         2
         Acacetin glycoside       1.00      1.00      1.00        16
              Acetaminophen       1.00      1.00      1.00         4
                Acetic acid       0.83      0.83      0.83        12
              Acetonic acid       0.60      1.00      0.75         3
             Acetyl choline       1.00      1.00      1.00         4
           Acetyl phosphate       1.00      1.00      1.00         1
              Aconitic acid       0.83      0.83      0.83         6
                  Aconitine       1.00      1.00      1.00         5
                    Adenine       1.00      1.00      1.00        30
                  Adenosine       0.98      1.00      0.99        56
        Adenosine phosphate       0.89      0.94      0.92        81
               Adinoxanthin      

D:\anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\anaconda2\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [53]:
pd.crosstab(y_test3,y_pred3, rownames=['Actual Species'], colnames=['Predicted Species'])

Predicted Species,Abscisic acid,Acacetin,Acacetin glycoside,Acetaminophen,Acetic acid,Acetonic acid,Acetyl choline,Acetyl phosphate,Aconitic acid,Aconitine,...,Zearalenone,Zeatin,Zeaxanthin,Zygadenin,caffeic acid,dimethoxycinnamic acid,ferulic acid,p-Coumaric acid,p-coumaric acid,tanshinone
Actual Species,,,,,,,,,,,,,,,,,,,,,
Abscisic acid,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acacetin,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acacetin glycoside,0,0,16,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetaminophen,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetic acid,0,0,0,0,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetonic acid,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetyl choline,0,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Acetyl phosphate,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Aconitic acid,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
